This one is getting ./AS13_reduc/data/AS13_stgdir_labelGeneric.csv

In [1]:
%%capture
!pip install codecarbon

In [2]:
#!pip uninstall wandb

In [3]:
# run this first cell and then do "restart and run all" from the run menu
!pip install -U accelerate

In [4]:
# Flaubert asks for this
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.8 MB/s eta 0:00:00


In [5]:
import os
os.environ["WANDB_MODE"] = "disabled"

##### CONFIG ####

dataset_name = "AS13_stgdir_labelGeneric"

model_dict = {1: "camembert-base", 2: "cmarkea/distilcamembert-base", 3: "bert-base-multilingual-cased", 4: "distilbert-base-multilingual-cased",
              5: "bert-base-multilingual-uncased", 6: "distilbert-base-multilingual-uncased",
              7: "flaubert/flaubert_base_cased", 8: "flaubert/flaubert_base_uncased"}

model_key = 7
model_ckpt = model_dict[model_key]
fraction_kept = 25 # can only be 100, 50, 25, 10, 5
run_number = 3

batch_size = 16
max_epochs = 40
assert not(fraction_kept < 50 and max_epochs < 10)

use_drive = False
use_drive_for_cm = True
save_model = False

# output dir format (below)
#output_dir=f"{model_ckpt}__finetuned__{dataset_name}__{fraction_kept}__{run_number}"

In [6]:
import os
if use_drive_for_cm:
  from google.colab import drive
  drive.mount("/content/drive")
  out_dir = "/content/drive/MyDrive/___tests_ft_d25"
  model_outname = model_ckpt.replace("/", "-")
  out_exp = os.path.join(f"stgcls_reps__{model_outname}__{str.zfill(str(fraction_kept), 3)}__{run_number}__{batch_size}")
  out_dir_exp = os.path.join(out_dir, out_exp)
  if not os.path.exists(out_dir_exp):
    os.mkdir(out_dir_exp)

Mounted at /content/drive


In [7]:
!ls /content/drive/MyDrive/___tests_ft_d25

d25
stgcls_reps__flaubert-flaubert_base_cased__010__1
stgcls_reps__flaubert-flaubert_base_cased__010__1__32
stgcls_reps__flaubert-flaubert_base_cased__010__2
stgcls_reps__flaubert-flaubert_base_cased__010__2__32
stgcls_reps__flaubert-flaubert_base_cased__010__3
stgcls_reps__flaubert-flaubert_base_cased__010__3__32
stgcls_reps__flaubert-flaubert_base_cased__010__4__32
stgcls_reps__flaubert-flaubert_base_cased__010__5__32
stgcls_reps__flaubert-flaubert_base_cased__025__1__16
stgcls_reps__flaubert-flaubert_base_cased__025__2__16
stgcls_reps__flaubert-flaubert_base_cased__025__3__16
stgcls_reps__flaubert-flaubert_base_cased__050__1__16
stgcls_reps__flaubert-flaubert_base_cased__050__2__16
stgcls_reps__flaubert-flaubert_base_cased__050__3__16


In [8]:
assert fraction_kept in [100*x for x in (1, 0.5, 0.25, 0.1, 0.05)]
assert run_number in (1, 2, 3, 4, 5)

In [9]:
# fraction kept to color map
clrmap_dict = {100: "Greens", 50: "Blues", 25: "Purples", 10: "Oranges", 5: "Reds"}
assert fraction_kept in clrmap_dict

In [10]:
!python --version

Python 3.10.12


In [11]:
!mkdir data

In [12]:
! pip install transformers

In [13]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [14]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [15]:
!pip install sentencepiece

In [16]:
import os

from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy
from transformers import FlaubertModel, FlaubertTokenizer
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
import evaluate
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
import torch

In [17]:
from sklearn.metrics import classification_report

J'ai téléversé les données sur mon serveur

In [18]:
![[ ! -d data ]] && mkdir data || echo "data directory exists"
#![[ ! -f data/AS13_stgdir_clean.csv ]] && wget -P data https://prf1.org/stgs/as-did/AS13_stgdir_clean.csv || echo "dataset already downloaded"
![[ ! -f data/AS13_stgdir_labelGeneric.csv ]] && wget -P data https://prf1.org/stgs/as-did/AS13_stgdir_labelGeneric.csv || echo "dataset already downloaded"

data directory exists
--2024-11-19 14:57:27--  https://prf1.org/stgs/as-did/AS13_stgdir_labelGeneric.csv
Resolving prf1.org (prf1.org)... 85.214.103.246
Connecting to prf1.org (prf1.org)|85.214.103.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1017742 (994K) [text/csv]
Saving to: ‘data/AS13_stgdir_labelGeneric.csv’

AS13_stgdir_labelGe 100%[===================>] 993.89K   924KB/s    in 1.1s    

2024-11-19 14:57:29 (924 KB/s) - ‘data/AS13_stgdir_labelGeneric.csv’ saved [1017742/1017742]



Recette pour éléchargement fichiers qui sont sur Drive sur la session Colab (sans monter le Drive) (pas nécessaire ici mais comme idée pour d'autres fichiers).
C'est un fichier quelconque non relié à la tâche qui a été utilisé pour tester.

In [19]:
mydf = "data/AS13_stgdir_labelGeneric.csv"

In [20]:
labelCol = 'labelGeneric'

In [21]:
import pandas as pd
AS13_df = pd.read_csv(mydf, sep="|", dtype={'description': 'object',
                                            labelCol: 'category'})

In [22]:
AS13_df = AS13_df.drop_duplicates()
AS13_df = AS13_df.dropna(how='any')
AS13_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14613 entries, 0 to 25821
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   description   14613 non-null  object  
 1   labelGeneric  14613 non-null  category
dtypes: category(1), object(1)
memory usage: 243.2+ KB


In [23]:
df = AS13_df

In [24]:
df['labelGeneric'].value_counts()

,count
labelGeneric,
music,2863
action,2467
toward,2144
exit,1295
object,1130
setting,982
delivery,962
entrance,646
movement,583


In [25]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(AS13_df[labelCol].unique().categories.to_list())
label2id = {class_names[i]:i for i in range(len(class_names))}
id2label = {i:class_names[i] for i in range(len(class_names))}

In [26]:
#class_names, label2id, id2label

In [27]:
# Paramètres

#batch_size = 32 # orig was 64 but I'm going down given CUDA OutOfMemory error
batch_size = 16 # for camembert-base, trying 16
# Proportion des données qui sera utilisée
scale = 0.2 # this is not used anywhere, can we remove it?

In [28]:
# nv dataFrame
data_df = pd.DataFrame()
data_df['text'] = AS13_df.description
# La classe cible est la région (province) sous forme d'identifiant numérique
data_df['label'] = AS13_df[labelCol].map(label2id)

In [29]:
data_df = data_df.reset_index(drop=True)

In [30]:
# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
stgtype_features = Features({'text': Value('string'),
                              'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=stgtype_features)


In [31]:
# Découpage en (train + validation) (this part will be reduced to 50%, 25% and so on to compare quality) et (test).
# The test part is supposed to always stay the same size and with the same examples

In [32]:
TRAIN_FULL, TRAIN_REDUCED = False, False
size_to_exclude = (100 - fraction_kept)/100
if size_to_exclude == 0:
  TRAIN_FULL = True
  print("FULL TRAIN")
else:
  TRAIN_REDUCED = True
  print(f"REDUCED TRAIN, KEEPING {fraction_kept}%")

REDUCED TRAIN, KEEPING 25%


In [33]:
print(f"SIZE TO EXCLUDE IS {size_to_exclude}")

SIZE TO EXCLUDE IS 0.75


In [34]:
if TRAIN_REDUCED:
  trainvalid_test = data.train_test_split(test_size=0.2, shuffle=True, seed=12)
  trainvalid_reduced = trainvalid_test['train'].train_test_split(test_size=size_to_exclude, shuffle=True, seed=12)
  trainvalid_reduced_trainvalid = trainvalid_reduced['train'].train_test_split(test_size=0.2, shuffle=True, seed=12)
  dataset = DatasetDict({"train": trainvalid_reduced_trainvalid["train"],
                         "valid": trainvalid_reduced_trainvalid["test"],
                         "test": trainvalid_test["test"]})
elif TRAIN_FULL:
  trainvalid_test = data.train_test_split(test_size=0.2, shuffle=True, seed=12)
  trainvalid_trainvalid = trainvalid_test["train"].train_test_split(test_size=0.2, shuffle=True, seed=12)
  dataset = DatasetDict({"train": trainvalid_trainvalid["train"],
                         "valid": trainvalid_trainvalid["test"],
                         "test": trainvalid_test["test"]})

In [35]:
data = dataset

In [36]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2337
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 585
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2923
    })
})

In [37]:
#steps = (epoch * examples)/batch size
total_steps = (6 * len(df))/16
total_steps

5479.875

In [38]:
# this is unused unless use steps as strategy in training args
epoch_size = len(data['train'])/batch_size

In [39]:
print(f"EPOCH SIZE IS: {epoch_size}")

EPOCH SIZE IS: 146.0625


When needed, export the dataset to verify later that it's indeed the same for all experiments

In [40]:
if False and use_drive:
  from google.colab import drive
  drive.mount("/content/drive")
  dataset_bkps = "/content/drive/MyDrive/___wv/datasets"
  current_dataset = "AS13_stgdir_labelGeneric_trainvalid_retest_uncased_25_pr_2"
  os.mkdir(os.path.join(dataset_bkps, current_dataset))
  for split, split_data in data.items():
    split_data.to_csv(Path(dataset_bkps) / current_dataset / f"{current_dataset}-{split}.csv")
    print(f"!head -n5 {split_path_str}")

In [41]:
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-train.csv
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-valid.csv
#!head -n5 /content/drive/MyDrive/___wv/datasets/AS13_stgdir_labelGeneric_trainvalid_retest_uncased/AS13_stgdir_labelGeneric_trainvalid_retest_uncased-test.csv

In [42]:
model_ckpt

'flaubert/flaubert_base_cased'

In [43]:
#!pip install sacremoses

In [44]:
if model_key in range(7,11):
  do_lc = False if model_key == 7 else True
  assert model_key in (7,8)
  tokenizer = FlaubertTokenizer.from_pretrained(model_ckpt, do_lowercase=do_lc)
else:
  # Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
  tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/896k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

In [45]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [46]:
%pdb

Automatic pdb calling has been turned ON


In [47]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=None)

Map:   0%|          | 0/2337 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

Map:   0%|          | 0/2923 [00:00<?, ? examples/s]

In [48]:
# Taille du vocabulaire
tokenizer.vocab_size

68729

In [49]:
# Taille de contexte maximum
tokenizer.model_max_length

512

In [50]:
accuracy = evaluate.load("accuracy")

In [51]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    return acc

In [52]:
#!curl ipinfo.io

In [53]:
from codecarbon import OfflineEmissionsTracker
tracker_before_train = OfflineEmissionsTracker(country_iso_code="FRA", save_to_file=True, output_dir=out_dir_exp,
                                               output_file=f"em_before_train__{os.path.basename(out_dir_exp)}.csv",
                                               log_level="critical")
tracker_before_train.start()

[codecarbon INFO @ 14:57:45] offline tracker init


In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if model_key in range(7000,1100):
  model = FlaubertModel.from_pretrained(model_ckpt).to(device)
else:
  model = AutoModelForSequenceClassification.from_pretrained(
      model_ckpt, num_labels=len(class_names), id2label=id2label, label2id=label2id
  ).to(device)

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
batch_size=16

In [56]:
from math import ceil
intended_eval_steps = ceil(len(data['train'])/batch_size)

In [57]:
# was recommended at https://stackoverflow.com/questions/69087044/early-stopping-in-bert-trainer-instances, but seems
# to no longer be necessary
# on 'eval_loss' as metric https://discuss.huggingface.co/t/early-stopping-training-using-validation-loss-as-the-metric-for-best-model/31378
if False:
  training_args = TrainingArguments(
      output_dir=f"{model_ckpt}__finetuned__{dataset_name}__{fraction_kept}__{run_number}",
      learning_rate=2e-5,
      num_train_epochs=max_epochs,
      evaluation_strategy=IntervalStrategy.STEPS,
      eval_steps = intended_eval_steps,
      save_steps = intended_eval_steps,
      save_total_limit = 3,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      weight_decay=0.01,
      save_strategy=IntervalStrategy.STEPS,
      load_best_model_at_end=True,
      metric_for_best_model='eval_loss',
  )

In [58]:
training_args = TrainingArguments(
    output_dir=f"{model_ckpt}__finetuned__{dataset_name}__{fraction_kept}__{run_number}",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=max_epochs,
    weight_decay=0.01,
    save_total_limit = 3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Now we use train et validation for the fine-tuning. We'll only use test to test the model after fine-tuning

In [59]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

<ipython-input-59-99445cd2dcae>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
tracker_before_train.stop()

/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


7.453045834671398e-06

In [61]:
batch_size

16

In [62]:
print(f"## FINE TUNING A {model_ckpt} MODEL ##")

## FINE TUNING A flaubert/flaubert_base_cased MODEL ##


In [63]:
tracker_train = OfflineEmissionsTracker(country_iso_code="FRA", save_to_file=True, output_dir=out_dir_exp,
                                        output_file=f"em_train__{os.path.basename(out_dir_exp)}.csv",
                                        log_level="critical")
tracker_train.start()

In [64]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.469101,0.570940
2,No log,0.758870,0.781197
3,No log,0.560409,0.808547
4,1.141800,0.546624,0.851282
5,1.141800,0.521221,0.868376
6,1.141800,0.642985,0.861538
7,0.254100,0.724832,0.851282
8,0.254100,0.761206,0.854701


TrainOutput(global_step=1176, training_loss=0.615620231952797, metrics={'train_runtime': 829.5067, 'train_samples_per_second': 112.693, 'train_steps_per_second': 7.089, 'total_flos': 4179555116408736.0, 'train_loss': 0.615620231952797, 'epoch': 8.0})

In [65]:
tracker_train.stop()

/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


0.001709667251657155

In [66]:
tracker_predict = OfflineEmissionsTracker(country_iso_code="FRA", save_to_file=True, output_dir=out_dir_exp,
                                          output_file=f"em_predict__{os.path.basename(out_dir_exp)}.csv",
                                          log_level="critical")
tracker_predict.start()

In [67]:
preds_output = trainer.predict(tokenized_data['test'])

In [68]:
tracker_predict.stop()

/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


9.444015979968336e-05

In [69]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [70]:
y_test = tokenized_data['test']['label']

In [71]:
labels = tokenized_data['test'].features['label'].names

In [72]:
# fraction kept to color map
clrmap_dict = {100: "Greens", 50: "Blues", 25: "Purples", 10: "Oranges", 5:"Reds"}

In [73]:
fraction_kept

25

In [74]:
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    cm_no_norm = confusion_matrix(y_true, y_preds)
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=labels_for_fig)
    #disp.plot(cmap="Purples", values_format=".2f", ax=ax, colorbar=False)
    disp.plot(cmap=clrmap_dict[fraction_kept], values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    return cm, cm_no_norm

cm_ob, cm_rawf_ob = plot_confusion_matrix(y_preds, y_test, labels)

In [75]:
print("Classification report:\n\n{}".format(classification_report(y_test, y_preds, digits=4)))

Classification report:

              precision    recall  f1-score   support

           0     0.8431    0.8292    0.8361       486
           1     0.7576    0.6667    0.7092        75
           2     1.0000    0.5000    0.6667        14
           3     0.8736    0.7136    0.7855       213
           4     0.7647    0.7109    0.7368       128
           5     0.8443    0.8512    0.8477       242
           6     0.7156    0.7647    0.7393       102
           7     0.6134    0.6134    0.6134       119
           8     0.9625    0.9341    0.9481       577
           9     0.6276    0.7583    0.6868       120
          10     0.7436    0.8365    0.7873       208
          11     0.7773    0.8632    0.8180       190
          12     0.9650    0.9822    0.9735       449

    accuracy                         0.8447      2923
   macro avg     0.8068    0.7711    0.7807      2923
weighted avg     0.8488    0.8447    0.8450      2923



In [76]:
for i in range(13):print(i, id2label[i], end=" ")

0 action 1 aggression 2 aparte 3 delivery 4 entrance 5 exit 6 interaction 7 movement 8 music 9 narration 10 object 11 setting 12 toward 

In [77]:
os.stat(out_dir_exp)

os.stat_result(st_mode=16832, st_ino=491, st_dev=87, st_nlink=2, st_uid=0, st_gid=0, st_size=4096, st_atime=1732028218, st_mtime=1732029150, st_ctime=1732028218)

In [78]:
import pickle

#cm_ob, cm_out_no_norm_ob = plot_confusion_matrix(y_preds, y_test, labels)

with open(os.path.join(out_dir_exp, f"cm_out___{os.path.basename(out_dir_exp)}.pkl"), mode="wb") as cm_fh:
  pickle.dump(cm_ob, cm_fh)
with open(os.path.join(out_dir_exp, f"cm_out_nonorm__{os.path.basename(out_dir_exp)}.pkl"), mode="wb") as cm_no_norm_fh:
  pickle.dump(cm_rawf_ob, cm_no_norm_fh)
with open(os.path.join(out_dir_exp, f"cr_out__{os.path.basename(out_dir_exp)}"), mode="w") as cr_out:
  cr_out.write(classification_report(y_test, y_preds, digits=4))

In [79]:
print(cm_ob)

[[0.82921811 0.01028807 0.         0.01028807 0.00205761 0.00205761
  0.01440329 0.02263374 0.00823045 0.03703704 0.04526749 0.01851852
  0.        ]
 [0.12       0.66666667 0.         0.         0.         0.
  0.01333333 0.         0.         0.05333333 0.13333333 0.01333333
  0.        ]
 [0.07142857 0.         0.5        0.14285714 0.         0.
  0.         0.14285714 0.         0.         0.         0.
  0.14285714]
 [0.07511737 0.00469484 0.         0.71361502 0.01877934 0.01408451
  0.03286385 0.00469484 0.03755869 0.05164319 0.01408451 0.00938967
  0.02347418]
 [0.         0.015625   0.         0.0078125  0.7109375  0.1484375
  0.         0.0234375  0.0234375  0.0234375  0.         0.046875
  0.        ]
 [0.02892562 0.         0.         0.00413223 0.04132231 0.85123967
  0.00826446 0.05371901 0.         0.         0.00413223 0.00826446
  0.        ]
 [0.01960784 0.02941176 0.         0.         0.01960784 0.
  0.76470588 0.03921569 0.         0.02941176 0.06862745 0.02941176

In [80]:
print(cm_rawf_ob)

[[403   5   0   5   1   1   7  11   4  18  22   9   0]
 [  9  50   0   0   0   0   1   0   0   4  10   1   0]
 [  1   0   7   2   0   0   0   2   0   0   0   0   2]
 [ 16   1   0 152   4   3   7   1   8  11   3   2   5]
 [  0   2   0   1  91  19   0   3   3   3   0   6   0]
 [  7   0   0   1  10 206   2  13   0   0   1   2   0]
 [  2   3   0   0   2   0  78   4   0   3   7   3   0]
 [  4   0   0   0   1  12   5  73   0   6   3  13   2]
 [  7   0   0   9   3   0   1   2 539   4   4   4   4]
 [  5   2   0   2   3   1   1   3   2  91   4   6   0]
 [ 17   3   0   2   2   0   7   0   1   1 174   1   0]
 [  4   0   0   0   2   2   0   6   2   3   4 164   3]
 [  3   0   0   0   0   0   0   1   1   1   2   0 441]]


In [81]:
if save_model:
  assert use_drive is True
  model.save_pretrained(f"/content/drive/MyDrive/___wv/{model_ckpt}-finetuned-stgdir-AS13reduc_50_common-testset")
  tokenizer.save_pretrained(f"/content/drive/MyDrive/___wv/{model_ckpt}-finetuned-stgdir-AS13reduc_50_common-testset")

In [82]:
!curl ipinfo.io

{
  "ip": "34.143.228.123",
  "hostname": "123.228.143.34.bc.googleusercontent.com",
  "city": "Singapore",
  "region": "Singapore",
  "country": "SG",
  "loc": "1.2897,103.8501",
  "org": "AS396982 Google LLC",
  "postal": "018989",
  "timezone": "Asia/Singapore",
  "readme": "https://ipinfo.io/missingauth"
}